In [0]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import RandomOverSampler as ROS
from imblearn.over_sampling import SMOTE as SMO
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train =pd.read_csv('/content/gdrive/My Drive/ML/Training_x_ad.csv',sep='\t', encoding='utf-8')
test = pd.read_csv('/content/gdrive/My Drive/ML/Testing_x_ad.csv',sep='\t', encoding='utf-8')

In [0]:
data = pd.concat([train,test],axis = 0)
data = data.reset_index()
data = data.drop(columns = ['index'])

In [0]:

data['ArrDelay'] = 0

def func(row):
    if row['ArrDel15']>0:# consider 15
        return 1
    else:
        return 0
data['ArrDelay'] = data.apply(func, axis=1)


In [0]:
x_tr = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek','CRSArrTime',
       'FlightNum', 'Origin', 'DestAirportID', 'Dest','year','DepDelay',
       'DewPointC', 'DewPointF', 'FeelsLikeC', 'FeelsLikeF', 'HeatIndexC',
       'HeatIndexF', 'WindChillC', 'WindChillF', 'WindGustKmph',
       'WindGustMiles', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'tempF', 'time', 'visibility', 'weatherCode', 'winddir16Point',
       'winddirDegree', 'windspeedKmph', 'windspeedMiles']
X= data[x_tr]
x_val = data[x_tr].iloc[:,:].values

In [0]:
x_val[0]

array([2.0, 5, 6, 6.0, 2139.0, 1185.0, 'ORD', 13303.0, 'MIA', 2017, -2.0,
       -1.0, 30.0, 11.0, 52.0, 11.0, 52.0, 11.0, 52.0, 25.0, 16.0, 10.0,
       40.0, 0.0, 1012.0, 11.0, 51.0, 1800.0, 10.0, 116.0, 'NNE', 31.0,
       23.0, 15.0], dtype=object)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
enc  = LabelEncoder()
enc.fit(x_val[:,8].reshape(-1, 1) )
c1=['d1']
temp = pd.DataFrame(enc.transform(x_val[:,6]),columns = c1)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Origin'])
c2=['a1']
temp = pd.DataFrame(enc.transform(x_val[:,8]),columns = c2)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Dest'])
X = X.drop(columns = ['winddir16Point'])
X = X.drop(columns = ['DewPointF','FeelsLikeC','FeelsLikeF','HeatIndexF','WindChillF','WindGustMiles','tempF','windspeedMiles','Quarter'])
x_val = X.iloc[:,:].values
y = data['ArrDelay']
y_val  = y.iloc[:].values

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
sampler = SMO()
X,y =  sampler.fit_resample(x_val,y_val)

In [0]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
from sklearn.model_selection import train_test_split 
X_train1, X_test1, y_train1, y_test1 = train_test_split(x_val, y_val, test_size=0.33, random_state=42)

In [0]:
sampler = SMO()
X_train1,y_train1 =  sampler.fit_resample(X_train1,y_train1)

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators=10)
classifier2.fit(X_train1,y_train1)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
print(classifier2.score(X_test1, y_test1, sample_weight=None))
from sklearn.metrics import classification_report
y_pred = classifier2.predict(X_test1)
print(classification_report(y_true=y_test1,y_pred = y_pred))

0.9120534895593257
              precision    recall  f1-score   support

           0       0.93      0.97      0.95    478773
           1       0.85      0.71      0.77    126504

    accuracy                           0.91    605277
   macro avg       0.89      0.84      0.86    605277
weighted avg       0.91      0.91      0.91    605277



In [0]:
y_pred = classifier2.predict(x_val)

In [0]:
y_pred

array([0, 1, 0, ..., 0, 0, 0])

In [0]:
data = data[y_pred==1]

In [0]:
data = data.reset_index()
data = data.drop(columns = ['index'])
data

,Unnamed: 0,Quarter,Month,DayofMonth,DayOfWeek,FlightNum,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,year,DewPointC,DewPointF,FeelsLikeC,FeelsLikeF,HeatIndexC,HeatIndexF,WindChillC,WindChillF,WindGustKmph,WindGustMiles,cloudcover,humidity,precipMM,pressure,tempC,tempF,time,visibility,weatherCode,winddir16Point,winddirDegree,windspeedKmph,windspeedMiles
0,629521,2.0,6,23,4.0,1121.0,IAH,12892.0,LAX,1615.0,0.0,69.0,69.0,1.0,1749.0,1842.0,53.0,1.0,2016,23.0,73.0,30.0,85.0,30.0,85.0,27.0,80.0,17.0,11.0,20.0,79.0,0.0,1017.0,27.0,80.0,0.0,10.0,113.0,SSE,165.0,11.0,7.0
1,1865969,4.0,11,9,4.0,851.0,SFO,12889.0,LAS,1240.0,0.0,22.0,22.0,1.0,1405.0,1422.0,17.0,1.0,2017,13.0,55.0,13.0,56.0,15.0,58.0,13.0,56.0,43.0,27.0,93.0,89.0,0.6,1013.0,15.0,58.0,0.0,8.0,353.0,S,189.0,22.0,14.0
2,595670,3.0,9,5,2.0,1998.0,EWR,11298.0,DFW,1920.0,22.0,150.0,150.0,1.0,2203.0,48.0,165.0,1.0,2017,20.0,69.0,26.0,78.0,26.0,78.0,24.0,74.0,16.0,10.0,84.0,83.0,0.0,1009.0,24.0,74.0,2200.0,10.0,122.0,SW,235.0,11.0,7.0
3,1183313,4.0,11,7,1.0,2505.0,MCO,11298.0,DFW,1715.0,0.0,132.0,132.0,1.0,1913.0,2128.0,135.0,1.0,2016,17.0,62.0,21.0,69.0,22.0,72.0,21.0,69.0,19.0,12.0,3.0,78.0,0.0,1021.0,21.0,69.0,0.0,10.0,113.0,NE,53.0,11.0,7.0
4,1072648,2.0,5,17,3.0,2503.0,LAX,13930.0,ORD,1125.0,12.0,35.0,35.0,1.0,1747.0,1838.0,51.0,1.0,2017,11.0,52.0,17.0,63.0,17.0,63.0,17.0,63.0,17.0,10.0,0.0,65.0,0.0,1013.0,17.0,62.0,1200.0,10.0,113.0,SSW,195.0,15.0,9.0
5,1818110,2.0,4,24,1.0,2755.0,SFO,12892.0,LAX,659.0,0.0,10.0,10.0,0.0,825.0,914.0,49.0,1.0,2017,8.0,47.0,13.0,55.0,13.0,55.0,13.0,55.0,25.0,16.0,67.0,78.0,0.0,1017.0,12.0,53.0,0.0,10.0,113.0,WNW,291.0,14.0,9.0
6,931551,1.0,1,2,6.0,6522.0,LAX,14107.0,PHX,1201.0,0.0,18.0,18.0,1.0,1428.0,1449.0,21.0,1.0,2016,-2.0,29.0,12.0,53.0,13.0,55.0,12.0,53.0,15.0,10.0,88.0,38.0,0.0,1020.0,13.0,55.0,0.0,10.0,119.0,NNE,33.0,9.0,6.0
7,1182064,4.0,10,10,1.0,998.0,MCO,12892.0,LAX,630.0,7.0,13.0,13.0,0.0,900.0,919.0,19.0,1.0,2016,16.0,61.0,20.0,67.0,20.0,67.0,20.0,67.0,27.0,17.0,23.0,81.0,0.0,1019.0,20.0,67.0,700.0,10.0,113.0,NNE,14.0,16.0,10.0
8,1472942,4.0,12,28,4.0,635.0,ORD,14747.0,SEA,915.0,12.0,163.0,163.0,1.0,1150.0,1501.0,191.0,1.0,2017,-18.0,-1.0,-19.0,-2.0,-12.0,10.0,-19.0,-2.0,15.0,9.0,100.0,61.0,0.0,1038.0,-12.0,10.0,1200.0,10.0,122.0,S,184.0,11.0,7.0
9,1749216,2.0,6,13,1.0,2777.0,SFO,12892.0,LAX,1845.0,19.0,-1.0,0.0,0.0,2025.0,2041.0,16.0,1.0,2016,13.0,55.0,15.0,60.0,16.0,60.0,15.0,60.0,31.0,19.0,84.0,82.0,0.0,1018.0,16.0,61.0,1900.0,10.0,119.0,WNW,282.0,27.0,17.0


In [0]:

x_tr = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek','CRSDepTime',
        'Origin',  'Dest','year','DepDelay',
       'DewPointC', 'DewPointF', 'FeelsLikeC', 'FeelsLikeF', 'HeatIndexC',
       'HeatIndexF', 'WindChillC', 'WindChillF', 'WindGustKmph',
       'WindGustMiles', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'tempF', 'time', 'visibility', 'weatherCode', 'winddir16Point',
       'winddirDegree', 'windspeedKmph', 'windspeedMiles']
X= data[x_tr]

In [0]:
def func(row):
    if(row['year']==2016):
      row['year'] = 1  # lable encoding the year values
    else:
      row['year'] = 2  
    return row
X= X.apply(func, axis=1)


In [0]:
x_val = X.iloc[:,:].values
x_val[0]


array([2.0, 6, 23, 4.0, 1615.0, 'IAH', 'LAX', 1, 69.0, 23.0, 73.0, 30.0,
       85.0, 30.0, 85.0, 27.0, 80.0, 17.0, 11.0, 20.0, 79.0, 0.0, 1017.0,
       27.0, 80.0, 0.0, 10.0, 113.0, 'SSE', 165.0, 11.0, 7.0],
      dtype=object)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
enc  = LabelEncoder()
enc.fit(x_val[:,6].reshape(-1, 1) )
c1=['d1']
temp = pd.DataFrame(enc.transform(x_val[:,5]),columns = c1)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Origin'])
c2=['a1']
temp = pd.DataFrame(enc.transform(x_val[:,6]),columns = c2)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Dest'])
X = X.drop(columns = ['winddir16Point'])
X = X.drop(columns = ['DewPointF','FeelsLikeC','FeelsLikeF','HeatIndexF','WindChillF','WindGustMiles','tempF','windspeedMiles','Quarter','time'])
x_val = X.iloc[:,:].values

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
Y = pd.DataFrame(data['ArrDelayMinutes'])
y_val = Y.iloc[:,:].values

In [0]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(x_val, y_val, test_size=0.33, random_state=42)

In [0]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train,y_train)
for i in zip(list(reg.coef_)[0],list(X.keys())):
  print(i)
print(reg.intercept_)

(0.0023068894899035746, 'Month')
(-0.0015501891538597379, 'DayofMonth')
(-0.046632968474268975, 'DayOfWeek')
(-0.003536231878063879, 'CRSDepTime')
(0.5468919366435214, 'year')
(0.9380249026515263, 'DepDelay')
(0.022142518595068304, 'DewPointC')
(0.4063071345187806, 'HeatIndexC')
(-0.33536635104739454, 'WindChillC')
(0.056216896178222925, 'WindGustKmph')
(0.010000690548520737, 'cloudcover')
(-0.016996577124406605, 'humidity')
(0.1472203260263827, 'precipMM')
(-0.12110690291572325, 'pressure')
(-0.07721624009458333, 'tempC')
(-0.044253311130122375, 'visibility')
(0.0051088984292017945, 'weatherCode')
(-0.001703606464288833, 'winddirDegree')
(-0.052697006316643986, 'windspeedKmph')
(-0.030252311204442498, 'd1')
(0.22812440069468512, 'a1')
[131.45532157]


In [0]:
from sklearn.metrics import mean_squared_error
import math
mse = mean_squared_error(y_test, reg.predict(X_test))
print("MSE: %.4f" % mse)
print("RMSE: %.4f" %math.sqrt(mse))
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, reg.predict(X_test))
print("MAE: %.4f" % mae)

MSE: 408.9991
RMSE: 20.2237
MAE: 14.9275


In [0]:
accuracy = reg.score(X_test,y_test)
print(accuracy*100,'%')

92.72300061055202 %
